# Multithreading y Multiprocessing

En este Notebook se mostrará la diferencia en tiempos de ejecución entre un programa escrito con la librería `multithreading` y la `multiprocessing`.

Para llevar a cabo este estudio se usará como aplicación ejemplo [El juego de la vida de Conway](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life).

## Conociendo el entorno

Inicialmente vamos a conocer cuantos núcleos nos ofrecen en este entorno de Notebook en Google Colab.

In [1]:
#
# Tomado de:
# - https://psutil.readthedocs.io/en/latest/#
# - https://superfastpython.com/number-of-cpus-python/#CPU_Count_with_multiprocessingcpu_count
#
import psutil

print(f'Numero de CPUs lógicas: {psutil.cpu_count()}')
print(f'Porcentaje de uso de la CPU {psutil.cpu_percent()}%')
print(f'Total memory {psutil.virtual_memory().total/(1024*1024):.2f} MB, porcentaje de memoria disponible {(psutil.virtual_memory().available / psutil.virtual_memory().total)*100:.2f}%')

Numero de CPUs lógicas: 8
Porcentaje de uso de la CPU 2.9%
Total memory 7818.03 MB, porcentaje de memoria disponible 85.70%


In [2]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 142
model name	: Intel(R) Core(TM) i7-8665U CPU @ 1.90GHz
stepping	: 12
microcode	: 0xffffffff
cpu MHz		: 2112.005
cache size	: 8192 KB
physical id	: 0
siblings	: 8
core id		: 0
cpu cores	: 4
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 22
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc arch_perfmon rep_good nopl xtopology cpuid pni pclmulqdq vmx ssse3 fma cx16 pdcm pcid sse4_1 sse4_2 movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp ibrs_enhanced tpr_shadow vnmi ept vpid ept_ad fsgsbase bmi1 avx2 smep bmi2 erms invpcid rdseed adx smap clflushopt xsaveopt xsavec xgetbv1 xsaves md_clear flush_l1d arch_capabilities
vmx flags	: vnmi invvpid ept_x_only ept_ad ept_1gb tsc_offset vtpr ept vpid unrestricted_guest ept_mode_ba

## Código del juego de la vida de Conway

En esta sección se verán algunas implementaciones del algoritmo del [juego de la vida de Conway](https://playgameoflife.com/).
Inicialmente se verá una versión secuencial, luego se introducirán los elementos para que esta versión secuencial se ejecute con el módulo de `multithreading` y luego `multiprocessing`.

En [Wikipedia](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life) se pueden encontrar las reglas que rigen el juego de la vida de Conway.

### Versión secuencial


In [7]:
# Version SECUENCIAL del juego de la vida de Conway
#
# Autor: John Sanabria - john.sanabria@correounivalle.edu.co
# Fecha: 2023-02-22

import numpy as np
from time import time

MAX_X = 750
MAX_Y = 750

#
# Funciones auxiliares usadas para estimar los vecinos de una celda y
# garantizar que los valores del borde no se vayan a sobrepasar
#
def posx(x, max_x = MAX_X):
  return (x + max_x) % max_x

def posy(y, max_y = MAX_Y):
  return (y + max_y) % max_y

def indice(x,y, max_x = MAX_X):
  return posx(x) + posy(y) * max_x

#
# Esta funcion se encarga de contar los 8 vecinos de una celda cuales están
# vivos (valor a '1') o cuantos están muertos (valor a '0')
#
#                 |                |
#  (X - 1, Y - 1) | (  X  , Y - 1) | (X + 1, Y - 1)
#                 |                |
# --------------------------------------------------
#                 |                |
#  (X - 1,   Y  ) | (  X  ,   Y  ) | (X + 1,   Y  )
#                 |                |
# --------------------------------------------------
#                 |                |
#  (X - 1, Y + 1) | (  X  , Y + 1) | (X + 1, Y + 1)
#                 |                |

def vecinos(x,y,vector):
  return vector[ indice(x - 1, y - 1) ] + vector[ indice(x - 1, y) ] + vector[ indice(x - 1, y + 1) ] \
  + vector[ indice(x, y - 1) ] + vector[ indice(x, y + 1) ] \
  + vector[ indice(x + 1, y - 1) ] + vector[ indice(x + 1, y) ] + vector[ indice(x + 1, y + 1) ]
#
# Esta función se encarga de recorrer toda las celdas y estimar cuales de ellas
# vivirán y cuales no
#
def conway(vector_out, vector_in, low_limit = 0, high_limit = MAX_X):
  for i in range(low_limit,high_limit):
    for j in range(0,MAX_Y):
      n = vecinos(i,j, vector_in)
      valor = 0
      if vector_in[ indice(i,j) ] == 1 and (n == 2 or n == 3):
        valor = 1
      elif vector_in[ indice(i,j) ] == 0 and n == 3:
        valor = 1
      vector_out[ indice(i,j) ] = valor

def print_vector_as_matrix(v):
  for i in range(0,MAX_X):
    for j in range(0,MAX_Y):
      print("%2d" % v[i*MAX_Y + j], end="")
    print()

#
# Función principal
#
if __name__ == '__main__':
  vector = np.int32( np.random.choice([1,0], MAX_X * MAX_Y, p = [0.50, 0.50]) )
  vector_out = np.empty(MAX_X * MAX_Y, dtype = np.int32)
  print(vector)
  t1 = time()
  conway(vector_out, vector)
  t2 = time()
  #print()
  print(vector_out)
  print(f"El tiempo que tomó calcular {t2 - t1}")

[0 0 1 ... 1 0 1]
[0 1 0 ... 0 0 0]
El tiempo que tomó calcular 4.61878228187561


### Versión multithreading

In [12]:
# Version MULTITHREADING del juego de la vida de Conway
#
# Autor: John Sanabria - john.sanabria@correounivalle.edu.co
# Fecha: 2023-02-22

import numpy as np
from threading import Thread
from multiprocessing import cpu_count
from time import time

MAX_UNITS = cpu_count() * 2
MAX_X = 750
MAX_Y = 750

#
# Funciones auxiliares usadas para estimar los vecinos de una celda y
# garantizar que los valores del borde no se vayan a sobrepasar
#

def posx(x, max_x = MAX_X):
  return (x + max_x) % max_x

def posy(y, max_y = MAX_Y):
  return (y + max_y) % max_y

def indice(x,y, max_x = MAX_X):
  return posx(x) + posy(y) * max_x

#
# Esta funcion se encarga de contar los 8 vecinos de una celda cuales están
# vivos (valor a '1') o cuantos están muertos (valor a '0')
#
#                 |                |
#  (X - 1, Y - 1) | (  X  , Y - 1) | (X + 1, Y - 1)
#                 |                |
# --------------------------------------------------
#                 |                |
#  (X - 1,   Y  ) | (  X  ,   Y  ) | (X + 1,   Y  )
#                 |                |
# --------------------------------------------------
#                 |                |
#  (X - 1, Y + 1) | (  X  , Y + 1) | (X + 1, Y + 1)
#

def vecinos(x,y,vector):
  return vector[ indice(x - 1, y - 1) ] + vector[ indice(x - 1, y) ] + vector[ indice(x - 1, y + 1) ] \
  + vector[ indice(x, y - 1) ] + vector[ indice(x, y + 1) ] \
  + vector[ indice(x + 1, y - 1) ] + vector[ indice(x + 1, y) ] + vector[ indice(x + 1, y + 1) ]
#
# Esta función se encarga de recorrer toda las celdas y estimar cuales de ellas
# vivirán y cuales no
#
def conway(vector_out, vector_in, low_limit = 0, high_limit = MAX_X):
  for i in range(low_limit,high_limit):
    for j in range(0,MAX_Y):
      n = vecinos(i,j, vector_in)
      valor = 0
      if vector_in[ indice(i,j) ] == 1 and (n == 2 or n == 3):
        valor = 1
      elif vector_in[ indice(i,j) ] == 0 and n == 3:
        valor = 1
      vector_out[ indice(i,j) ] = valor

#
# Función principal
#
if __name__ == '__main__':
  vector = np.int32( np.random.choice([1,0], MAX_X * MAX_Y, p = [0.50, 0.50]) )
  vector_out = np.empty(MAX_X * MAX_Y, dtype = np.int32)
  print(vector)
  # Variables usadas para hacer la partición de las filas que procesara cada
  # hilo
  stride = np.int32(MAX_X  / MAX_UNITS)
  count = np.int32(0)
  workers = [] # arreglo que almacenará aquellos hilos que se ejecutan
  for t in range(MAX_UNITS):
    worker = Thread(target = conway,
                    args = (vector_out, vector, count, count + stride))
    worker.daemon = True
    workers.append(worker)
    count = count + stride

  t1 = time()
  for worker in workers:
    worker.start()
  # Ciclo que espera por la terminación de todos los hilos que se lanzaron
  for worker in workers:
    worker.join()
  t2 = time()
  print(vector_out)
  print(f"El tiempo que tomó calcular {t2 - t1}")

[1 0 0 ... 1 1 1]
[0 0 0 ... 1 1 0]
El tiempo que tomó calcular 4.4988343715667725


### Versión multiprocessing

In [18]:
# Version MULTIPROCESSING del juego de la vida de Conway
#
# Autor: John Sanabria - john.sanabria@correounivalle.edu.co
# Fecha: 2023-02-22
#
# Modificado
#
# 2024-02-14: generacion aleatoria de un punto en el mundo de Conway

import numpy as np
from multiprocessing import Process
from multiprocessing import cpu_count
from multiprocessing import RawArray
from time import time
import random

MAX_UNITS = cpu_count()
MAX_X = 750
MAX_Y = 750

_vector = np.int32( np.random.choice([1,0], MAX_X * MAX_Y, p = [0.50, 0.50]) )
vector = RawArray('i',_vector)
_vector_out = np.empty(MAX_X * MAX_Y, dtype = np.int32)
vector_out = RawArray('i',_vector_out)

#
# Funciones auxiliares usadas para estimar los vecinos de una celda y
# garantizar que los valores del borde no se vayan a sobrepasar
#

def posx(x, max_x = MAX_X):
  return (x + max_x) % max_x

def posy(y, max_y = MAX_Y):
  return (y + max_y) % max_y

def indice(x,y, max_y = MAX_Y):
  return posx(x) * max_y + posy(y)

#
# Esta funcion se encarga de contar los 8 vecinos de una celda cuales están
# vivos (valor a '1') o cuantos están muertos (valor a '0')
#
#                 |                |
#  (X - 1, Y - 1) | (  X  , Y - 1) | (X + 1, Y - 1)
#                 |                |
# --------------------------------------------------
#                 |                |
#  (X - 1,   Y  ) | (  X  ,   Y  ) | (X + 1,   Y  )
#                 |                |
# --------------------------------------------------
#                 |                |
#  (X - 1, Y + 1) | (  X  , Y + 1) | (X + 1, Y + 1)
#

def vecinos(x,y):
  return vector[ indice(x - 1, y - 1) ] + vector[ indice(x - 1 , y) ] + vector[ indice(x - 1, y + 1) ] \
  + vector[ indice(x, y - 1) ] + vector[ indice(x, y + 1) ] \
  + vector[ indice(x + 1, y - 1) ] + vector[ indice(x + 1 , y) ] + vector[ indice(x + 1, y + 1) ]

def imprimir_vecinos(x,y):
    print(f"La posicion ({x},{y}) paso de {vector[indice(x,y)]} a {vector_out[indice(x,y)]}")
    print("Sus vecinos eran")
    print(f"{vector[ indice(x - 1, y - 1) ]}, {vector[ indice(x - 1, y) ]}, {vector[ indice(x - 1, y + 1)]}")
    print(f"{vector[ indice(x, y - 1) ]}, X, {vector[ indice(x, y + 1) ]}")
    print(f"{vector[ indice(x + 1, y - 1) ]}, {vector[ indice(x + 1, y) ]}, {vector[ indice(x + 1, y + 1) ]}")

#
# Esta función se encarga de recorrer toda las celdas y estimar cuales de ellas
# vivirán y cuales no
#
def conway(low_limit = 0, high_limit = MAX_X):
  #print(f"De fila {low_limit} a fila {high_limit}")
  for i in range(low_limit,high_limit):
    for j in range(0,MAX_Y):
      n = vecinos(i,j)
      valor = 0
      if vector[ indice(i,j) ] == 1 and (n == 2 or n == 3):
        valor = 1
      elif vector[ indice(i,j) ] == 0 and n == 3:
        valor = 1
      vector_out[ indice(i,j) ] = valor

#
# Función principal
#
if __name__ == '__main__':
  #print(np.frombuffer(vector, dtype = np.int32))
  print(np.frombuffer(vector, dtype = np.int32).reshape(MAX_X, MAX_Y))
  # Variables usadas para hacer la partición de las filas que trabajará cada
  # proceso
  stride = np.int32(MAX_X / MAX_UNITS)
  count = np.int32(0)
  workers = [] # arreglo que almacenará aquellos procesos que se crearán
  for t in range(MAX_UNITS):
    worker = Process(target = conway,
                    args = (count, np.int32(count + stride)))
    workers.append(worker)
    #print(count)
    count = count + stride
  # Ciclo que lanza la ejecución de los procesos que se crearon
  t1 = time()
  for worker in workers:
    worker.start()
  # Ciclo que espera por la terminación de todos los procesos que se lanzaron
  for worker in workers:
    worker.join()
  t2 = time()
  print()
  print(np.frombuffer(vector_out, dtype = np.int32).reshape(MAX_X, MAX_Y))
  print(f"El tiempo que tomó calcular {t2 - t1}")
  imprimir_vecinos(random.randint(0,MAX_X), random.randint(0,MAX_Y))

[[0 0 0 ... 1 0 1]
 [1 1 1 ... 1 1 1]
 [0 0 1 ... 1 1 1]
 ...
 [0 1 0 ... 0 0 0]
 [0 1 1 ... 1 1 0]
 [1 1 0 ... 0 1 1]]

[[         0          0          0 ...          0          0          0]
 [         0          1          1 ...          0          0          0]
 [         0          0          0 ...          0          0          0]
 ...
 [-450534128      32553 -450534096 ...      32553 -450522096      32553]
 [-450522064      32553 -450522032 ...      32553 -450510032      32553]
 [-450510000      32553 -450509968 ...      32553 -450498032      32553]]
El tiempo que tomó calcular 0.6754188537597656
La posicion (467,704) paso de 0 a 0
Sus vecinos eran
0, 1, 1
1, X, 1
1, 0, 1
